<p style="font-family: Arial; font-size:2.75em;color:purple; font-style:bold"><br>

Regresssion with scikit-learn<br><br> using Soccer Dataset

<br><br></p>


We will again be using the open dataset from the popular site <a href="https://www.kaggle.com">Kaggle</a> that we used in Week 1 for our example. 

Recall that this <a href="https://www.kaggle.com/hugomathien/soccer">European Soccer Database</a> has more than 25,000 matches and more than 10,000 players for European professional soccer seasons from 2008 to 2016. 

**Note:** Please download the file *database.sqlite* if you don't yet have it in your *Week-7-MachineLearning* folder.

<p style="font-family: Arial; font-size:1.75em;color:purple; font-style:bold"><br>

Import Libraries<br><br></p>


In [1]:
import sqlite3
import pandas as pd 
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt

<p style="font-family: Arial; font-size:1.75em;color:purple; font-style:bold"><br>

Read Data from the Database into pandas
<br><br></p>


In [2]:
# Create your connection.
cnx = sqlite3.connect('database.sqlite')
df = pd.read_sql_query("SELECT * FROM Player_Attributes", cnx)

In [22]:
df[df["gk_handling"] >60].head()

,id,player_fifa_api_id,player_api_id,date,overall_rating,potential,preferred_foot,attacking_work_rate,defensive_work_rate,crossing,finishing,heading_accuracy,short_passing,volleys,dribbling,curve,free_kick_accuracy,long_passing,ball_control,acceleration,sprint_speed,agility,reactions,balance,shot_power,jumping,stamina,strength,long_shots,aggression,interceptions,positioning,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
695,696,197233,189181,2016-04-21 00:00:00,69.0,74.0,right,medium,medium,12.0,11.0,13.0,34.0,14.0,15.0,15.0,14.0,25.0,17.0,42.0,40.0,57.0,71.0,52.0,19.0,66.0,31.0,75.0,12.0,18.0,24.0,11.0,24.0,20.0,19.0,20.0,12.0,70.0,63.0,63.0,70.0,75.0
696,697,197233,189181,2016-03-31 00:00:00,69.0,75.0,right,medium,medium,12.0,11.0,13.0,34.0,14.0,15.0,15.0,14.0,25.0,17.0,42.0,40.0,57.0,71.0,52.0,19.0,66.0,31.0,75.0,12.0,18.0,24.0,11.0,24.0,20.0,19.0,20.0,12.0,70.0,63.0,63.0,70.0,75.0
697,698,197233,189181,2016-01-07 00:00:00,69.0,76.0,right,medium,medium,12.0,11.0,13.0,34.0,14.0,15.0,15.0,14.0,25.0,17.0,42.0,40.0,57.0,71.0,52.0,19.0,66.0,31.0,75.0,12.0,18.0,24.0,11.0,24.0,20.0,19.0,20.0,12.0,70.0,63.0,63.0,70.0,75.0
698,699,197233,189181,2015-10-02 00:00:00,69.0,76.0,right,medium,medium,12.0,11.0,13.0,34.0,14.0,15.0,15.0,14.0,25.0,17.0,42.0,40.0,57.0,71.0,52.0,19.0,66.0,31.0,75.0,12.0,18.0,24.0,11.0,24.0,20.0,19.0,20.0,12.0,70.0,63.0,63.0,70.0,75.0
699,700,197233,189181,2015-09-21 00:00:00,67.0,74.0,right,medium,medium,12.0,11.0,13.0,34.0,14.0,15.0,15.0,14.0,25.0,17.0,42.0,32.0,57.0,71.0,52.0,19.0,66.0,31.0,75.0,12.0,18.0,24.0,11.0,24.0,20.0,19.0,20.0,12.0,66.0,63.0,62.0,65.0,75.0


In [4]:
df.shape

(183978, 42)

In [23]:
df.columns

Index(['id', 'player_fifa_api_id', 'player_api_id', 'date', 'overall_rating',
       'potential', 'preferred_foot', 'attacking_work_rate',
       'defensive_work_rate', 'crossing', 'finishing', 'heading_accuracy',
       'short_passing', 'volleys', 'dribbling', 'curve', 'free_kick_accuracy',
       'long_passing', 'ball_control', 'acceleration', 'sprint_speed',
       'agility', 'reactions', 'balance', 'shot_power', 'jumping', 'stamina',
       'strength', 'long_shots', 'aggression', 'interceptions', 'positioning',
       'vision', 'penalties', 'marking', 'standing_tackle', 'sliding_tackle',
       'gk_diving', 'gk_handling', 'gk_kicking', 'gk_positioning',
       'gk_reflexes'],
      dtype='object')

<p style="font-family: Arial; font-size:1.75em;color:purple; font-style:bold"><br>

Declare the Columns You Want to Use as Features
<br><br></p>


In [59]:
features = [
       'potential', 'crossing', 'finishing', 'heading_accuracy',
       'short_passing', 'volleys', 'dribbling', 'curve', 'free_kick_accuracy',
       'long_passing', 'ball_control', 'acceleration', 'sprint_speed',
       'agility', 'reactions', 'balance', 'shot_power', 'jumping', 'stamina',
       'strength', 'long_shots', 'aggression', 'interceptions', 'positioning',
       'vision', 'penalties', 'marking', 'standing_tackle', 'sliding_tackle',
       'gk_diving', 'gk_handling', 'gk_kicking', 'gk_positioning',
       'gk_reflexes']


AttributeError: 'list' object has no attribute 'sum'

<p style="font-family: Arial; font-size:1.75em;color:purple; font-style:bold"><br>

Specify the Prediction Target
<br><br></p>


In [25]:
target = ['overall_rating']

<p style="font-family: Arial; font-size:1.75em;color:purple; font-style:bold"><br>

Clean the Data<br><br></p>


In [36]:
df = df.dropna()

<p style="font-family: Arial; font-size:1.75em;color:purple; font-style:bold"><br>

Extract Features and Target ('overall_rating') Values into Separate Dataframes
<br><br></p>


In [37]:
X = df[features]

In [38]:
y = df[target]

Let us look at a typical row from our features: 

In [56]:
X.iloc[0].describe()

count    34.000000
mean     49.852941
std      20.191706
min       6.000000
25%      44.250000
50%      54.000000
75%      64.750000
max      76.000000
Name: 0, dtype: float64

Let us also display our target values: 

In [54]:
y


,overall_rating
0,67.0
1,67.0
2,62.0
3,61.0
4,61.0
5,74.0
6,74.0
7,73.0
8,73.0
9,73.0


<p style="font-family: Arial; font-size:1.75em;color:purple; font-style:bold"><br>

Split the Dataset into Training and Test Datasets
<br><br></p>


In [60]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=324)

<p style="font-family: Arial; font-size:1.75em;color:purple; font-style:bold"><br>

(1) Linear Regression: Fit a model to the training set
<br><br></p>


In [61]:
regressor = LinearRegression()
regressor.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

<p style="font-family: Arial; font-size:1.75em;color:purple; font-style:bold"><br>

Perform Prediction using Linear Regression Model
<br><br></p>


In [62]:
y_prediction = regressor.predict(X_test)
y_prediction

array([[ 66.51284879],
       [ 79.77234615],
       [ 66.57371825],
       ..., 
       [ 69.23780133],
       [ 64.58351696],
       [ 73.6881185 ]])

<p style="font-family: Arial; font-size:1.75em;color:purple; font-style:bold"><br>

What is the mean of the expected target value in test set ?
<br><br></p>


In [63]:
y_test.describe()

,overall_rating
count,59517.000000
mean,68.635818
std,7.041297
min,33.000000
25%,64.000000
50%,69.000000
75%,73.000000
max,94.000000


<p style="font-family: Arial; font-size:1.75em;color:purple; font-style:bold"><br>

Evaluate Linear Regression Accuracy using Root Mean Square Error

<br><br></p>


In [64]:
RMSE = sqrt(mean_squared_error(y_true = y_test, y_pred = y_prediction))

In [65]:
print(RMSE)

2.805303046855209


<p style="font-family: Arial; font-size:1.75em;color:purple; font-style:bold"><br>

(2) Decision Tree Regressor: Fit a new regression model to the training set
<br><br></p>


In [98]:
regressor_tree = DecisionTreeRegressor(max_depth=20)
regressor_tree.fit(X_train, y_train)

DecisionTreeRegressor(criterion='mse', max_depth=20, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best')

<p style="font-family: Arial; font-size:1.75em;color:purple; font-style:bold"><br>

Perform Prediction using Decision Tree Regressor
<br><br></p>


In [99]:
y_prediction_tree = regressor_tree.predict(X_test)
y_prediction_tree

array([ 62.        ,  84.        ,  62.38666667, ...,  71.        ,
        62.        ,  73.        ])

<p style="font-family: Arial; font-size:1.75em;color:purple; font-style:bold"><br>

For comparision: What is the mean of the expected target value in test set ?
<br><br></p>


In [68]:
y_test.describe()

,overall_rating
count,59517.000000
mean,68.635818
std,7.041297
min,33.000000
25%,64.000000
50%,69.000000
75%,73.000000
max,94.000000


<p style="font-family: Arial; font-size:1.75em;color:purple; font-style:bold"><br>

Evaluate Decision Tree Regression Accuracy using Root Mean Square Error

<br><br></p>


In [100]:
RMSE2 = sqrt(mean_squared_error(y_true = y_test, y_pred = y_prediction_tree))

In [101]:
print(RMSE2)

1.4517548844489676
